<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/Multi_Agent_System_for_Medical_Literature_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [2]:
import os

os.environ['OPENAI_API_KEY'] = ''


In [3]:
import os
import uuid
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Agent Base Class
class Agent:
    def __init__(self, agent_id=None):
        self.agent_id = agent_id or str(uuid.uuid4())
        self.role = 'Agent'
        self.memory = []
        self.peers = []
        self.knowledge_base = None  # Reference to shared knowledge base
        self.communication_channel = None  # Reference to communication channel

    def communicate(self, message, recipient_id=None):
        if self.communication_channel:
            self.communication_channel.send(self.agent_id, recipient_id, message)

    def receive_messages(self):
        if self.communication_channel:
            return self.communication_channel.receive(self.agent_id)
        return []

    def query_gpt(self, prompt):
        response = client.chat.completions.create(
            model='gpt-4',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content

# Specialized Agents
class ResearchAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Research Agent'

    def perform_task(self, topic):
        prompt = f"""
You are a medical research assistant. Gather the latest research articles, studies, and clinical trials related to "{topic}". Provide citations and brief summaries of each source.
"""
        research_data = self.query_gpt(prompt)
        self.memory.append({'task': 'research', 'data': research_data})
        return research_data

class AnalysisAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Analysis Agent'

    def perform_task(self, research_data):
        prompt = f"""
You are a medical data analyst. Analyze the following research data and extract key findings, trends, and significant results.

Research Data:
{research_data}
"""
        analysis = self.query_gpt(prompt)
        self.memory.append({'task': 'analysis', 'data': analysis})
        return analysis

class SummarizationAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Summarization Agent'

    def perform_task(self, analysis):
        prompt = f"""
You are a medical writer. Summarize the following analysis into a comprehensive report suitable for medical professionals.

Analysis:
{analysis}
"""
        summary = self.query_gpt(prompt)
        self.memory.append({'task': 'summarization', 'data': summary})
        return summary

class VerificationAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Verification Agent'

    def perform_task(self, summary):
        prompt = f"""
You are a medical editor. Verify the following summary for accuracy, clarity, and adherence to medical guidelines. Suggest corrections if necessary.

Summary:
{summary}
"""
        verification = self.query_gpt(prompt)
        self.memory.append({'task': 'verification', 'data': verification})
        return verification

class EthicalComplianceAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Ethical Compliance Agent'

    def perform_task(self, verified_summary):
        prompt = f"""
You are responsible for ensuring that the following medical report complies with all ethical guidelines and policies, including patient privacy and data protection. Confirm compliance or highlight any issues.

Report:
{verified_summary}
"""
        compliance = self.query_gpt(prompt)
        self.memory.append({'task': 'ethical_compliance', 'data': compliance})
        return compliance

# Communication Channel
class CommunicationChannel:
    def __init__(self):
        self.messages = []

    def send(self, sender_id, recipient_id, content):
        message = {'sender': sender_id, 'recipient': recipient_id, 'content': content}
        self.messages.append(message)

    def receive(self, agent_id):
        agent_messages = [
            msg for msg in self.messages
            if msg['recipient'] == agent_id or msg['recipient'] is None
        ]
        # Remove the messages once received
        self.messages = [msg for msg in self.messages if msg not in agent_messages]
        return agent_messages

# Knowledge Base (Simplified for this example)
class KnowledgeBase:
    def __init__(self):
        self.data = {}

    def store(self, key, value):
        self.data[key] = value

    def retrieve(self, key):
        return self.data.get(key, None)

# Initialize shared components
knowledge_base = KnowledgeBase()
communication_channel = CommunicationChannel()

# Create agents
research_agent = ResearchAgent()
analysis_agent = AnalysisAgent()
summarization_agent = SummarizationAgent()
verification_agent = VerificationAgent()
ethical_agent = EthicalComplianceAgent()

agents = [
    research_agent,
    analysis_agent,
    summarization_agent,
    verification_agent,
    ethical_agent,
]

# Assign shared components to agents
for agent in agents:
    agent.knowledge_base = knowledge_base
    agent.communication_channel = communication_channel
    agent.peers = [a.agent_id for a in agents if a.agent_id != agent.agent_id]

# Define the medical topic
medical_topic = "The latest advancements in immunotherapy for cancer treatment"

# Workflow Execution
def execute_workflow(topic):
    # Step 1: Research
    print(f"\n{research_agent.role} is performing research.")
    research_data = research_agent.perform_task(topic)
    research_agent.communicate(research_data, analysis_agent.agent_id)

    # Step 2: Analysis
    print(f"\n{analysis_agent.role} is analyzing research data.")
    messages = analysis_agent.receive_messages()
    if messages:
        analysis_input = messages[0]['content']
        analysis = analysis_agent.perform_task(analysis_input)
        analysis_agent.communicate(analysis, summarization_agent.agent_id)

    # Step 3: Summarization
    print(f"\n{summarization_agent.role} is summarizing analysis.")
    messages = summarization_agent.receive_messages()
    if messages:
        summarization_input = messages[0]['content']
        summary = summarization_agent.perform_task(summarization_input)
        summarization_agent.communicate(summary, verification_agent.agent_id)

    # Step 4: Verification
    print(f"\n{verification_agent.role} is verifying the summary.")
    messages = verification_agent.receive_messages()
    if messages:
        verification_input = messages[0]['content']
        verified_summary = verification_agent.perform_task(verification_input)
        verification_agent.communicate(verified_summary, ethical_agent.agent_id)

    # Step 5: Ethical Compliance
    print(f"\n{ethical_agent.role} is checking for ethical compliance.")
    messages = ethical_agent.receive_messages()
    if messages:
        compliance_input = messages[0]['content']
        compliance_result = ethical_agent.perform_task(compliance_input)
        ethical_agent.communicate(compliance_result)

    # Final Output
    print("\nFinal Report:")
    print(compliance_input)
    print("\nEthical Compliance Confirmation:")
    print(compliance_result)

# Execute the workflow
execute_workflow(medical_topic)



Research Agent is performing research.

Analysis Agent is analyzing research data.

Summarization Agent is summarizing analysis.

Verification Agent is verifying the summary.

Ethical Compliance Agent is checking for ethical compliance.

Final Report:
Summary:
The report provides an overview of recent advancements in cancer immunotherapy that have shown promising results. They include immune checkpoint discovery, genetic and cellular therapeutics, Chimeric Antigen Receptor T (CAR T) cell therapy, and Bi-specific T-cell engagers (BiTEs). 

Immunotherapy is reported as being less toxic than traditional treatments, potentially causing fewer side effects and improving patient prognosis and quality of life. The combination of natural and synthetic compounds may enhance immunotherapy effects, with personalization of treatment combinations optimally tailored to specific patient conditions being recognized as the potential future of this therapy. 

The use of the CRISPR/Cas9 gene-editing tech